The recommendation for the user’s splash/home page can be created from either python or Azure ML Studio. An “adequate” fulfillment of this requirement would be to create a single list of movie recommendations. A “good” solution would be to create multiple lists that are generated in different ways. For example, you could have one list based on collaborative filtering where the movies are based on the top n rated movies from the user and another list is based on content filtering. A “best” solution would surprise us in some way. For example, you could filter the recommendations by genre and provide separate lists of recommended “Action”, “Comedy”, and “Childrens” movies



In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

df_movies_titles = pd.read_csv(r"C:\Users\camde\Downloads\movies_titles.csv")
df_user_ratings = pd.read_csv(r"C:\Users\camde\Downloads\movies_ratings.csv")

DATA UNDERSTANDING

In [2]:
df_movies_titles.head()
df_movies_titles['show_id'] = df_movies_titles['show_id'].str.lstrip('s')

df_movies_titles.set_index('show_id', inplace=True)
df_movies_titles.head()

,type,title,director,cast,country,release_year,rating,duration,description,Action,...,Language TV Shows,Musicals,Nature TV,Reality TV,Spirituality,TV Action,TV Comedies,TV Dramas,Talk Shows TV Comedies,Thrillers
show_id,,,,,,,,,,,,,,,,,,,,,
1,Movie,Dick Johnson Is Dead,Kirsten Johnson,"Michael Hilow, Ana Hoffman, Dick Johnson, Kirs...",United States,2020,PG-13,90 min,As her father nears the end of his life filmma...,0,...,0,0,0,0,0,0,0,0,0,0
2,TV Show,Blood & Water,NaN,Ama Qamata Khosi Ngema Gail Mabalane Thabang M...,South Africa,2021,TV-MA,2 Seasons,After crossing paths at a party a Cape Town te...,0,...,0,0,0,0,0,0,0,1,0,0
3,TV Show,Ganglands,Julien Leclercq,Sami Bouajila Tracy Gotoas Samuel Jouy Nabiha ...,NaN,2021,TV-MA,1 Season,To protect his family from a powerful drug lor...,1,...,0,0,0,0,0,1,0,0,0,0
4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021,TV-MA,1 Season,Feuds flirtations and toilet talk go down amon...,0,...,0,0,0,1,0,0,0,0,0,0
5,TV Show,Kota Factory,NaN,Mayur More Jitendra Kumar Ranjan Raj Alam Khan...,India,2021,TV-MA,2 Seasons,In a city of coaching centers known to train I...,0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
def univariate_stats(df, roundto = 4):
    import pandas as pd
    import numpy as np

    df_results = pd.DataFrame(columns=['dtype', 'count', 'missing', 'unique', 'mode', 
                                       'min', 'q1', 'median', 'q3', 'max', 
                                       'mean', 'std', 'skew', 'kurt'])
    
    for col in df:
        dtype = df[col].dtype
        count = df[col].count()
        missing = df[col].isna().sum()
        unique = df[col].nunique()
        try:
            mode = df[col].mode()[0]
        except:
            print(f"Mode cannot be determined for {col}")
            mode = np.nan
        
        if pd.api.types.is_numeric_dtype(df[col]):
            min = df[col].min()
            q1 = df[col].quantile(0.25)
            median = df[col].median()
            q3 = df[col].quantile(0.75)
            max = df[col].max()
            mean = df[col].mean()
            std = df[col].std()
            skew = df[col].skew()
            kurt = df[col].kurt()

            df_results.loc[col] = [dtype, count, missing, unique, mode, 
                                   round(min, roundto), round(q1, roundto), round(median, roundto),
                                    round(q3, roundto), round(max, roundto), round(mean, roundto), 
                                    round(std, roundto), round(skew, roundto), round(kurt, roundto)]
        
        else:
            df_results.loc[col] = [dtype, count, missing, unique, mode, "", "", "", "", "", "", "", "", ""]


    return df_results

In [7]:
univariate_stats(df_movies_titles)

,dtype,count,missing,unique,mode,min,q1,median,q3,max,mean,std,skew,kurt
type,object,8508,0,2,Movie,,,,,,,,,
title,object,8508,0,8505,15-Aug,,,,,,,,,
director,object,6394,2114,4703,Rajiv Chilaka,,,,,,,,,
cast,object,8389,119,8117,David Attenborough,,,,,,,,,
country,object,7694,814,731,United States,,,,,,,,,
release_year,int64,8508,0,74,2018,1925,2013.0,2017.0,2019.0,2021,2014.1861,8.8365,-3.466,16.4059
rating,object,8504,4,14,TV-MA,,,,,,,,,
duration,object,8507,1,220,1 Season,,,,,,,,,
description,object,8508,0,8476,Paranormal activity at a lush abandoned proper...,,,,,,,,,
Action,int64,8508,0,2,0,0,0.0,0.0,0.0,1,0.1195,0.3244,2.346,3.5043


DATA PREPARATION

In [ ]:
# strip the s off the id
df_user_ratings['show_id'] = df_user_ratings['show_id'].str.lstrip('s')
df_user_ratings.head()


,user_id,show_id,rating
0,1,8381,4
1,1,3466,5
2,1,3181,4
3,1,294,3
4,1,7739,5


collaborative filtering model (top 3 movies the user has rated)

MODELING AND EVALUATION

In [11]:
# Aggregating interactions (if a user has multiple interactions, i.e. VIEW and COMMENT, and LIKE
# it will take the rating with the highest value since that shows the highest LEVEL of interaction
# that the user actually had)
df_ratings = df_user_ratings.groupby(['user_id', 'show_id'], as_index=False)['rating'].max()

In [12]:
# Create the User Item matrix (faster since it's a HUGE dataset of possibilities)

num_users = df_ratings['user_id'].nunique()
num_items = df_ratings['show_id'].nunique()

In [13]:
# Create mappings from the actual IDs to a range of integers (making the rating coordinates
# again, the spaces/empty values are inferred based on these coordinates I think)
user_mapper = dict(zip(np.sort(df_ratings['user_id'].unique()), range(num_users)))
item_mapper = dict(zip(np.sort(df_ratings['show_id'].unique()), range(num_items)))

In [14]:
# Reverse mappings
user_inv_mapper = {v: k for k, v in user_mapper.items()}
item_inv_mapper = {v: k for k, v in item_mapper.items()}

In [15]:
# Map the IDs to indices in the dataframe
df_ratings['user_index'] = df_ratings['user_id'].map(user_mapper)
df_ratings['item_index'] = df_ratings['show_id'].map(item_mapper)

In [16]:
# Creating the CSR matrix
user_item_matrix = csr_matrix(
    (df_ratings['rating'], (df_ratings['item_index'], df_ratings['user_index'])),
    shape=(num_items, num_users)
)

In [17]:
# Build the Collaborative Filtering Model
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
model.fit(user_item_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [18]:
def recommend_articles(contentId, model, X, item_mapper, item_inv_mapper, k=5, metric='cosine', messages=False):
    import numpy as np
    # Look up the article index using the mapping
    article_index = item_mapper.get(contentId)
    if article_index is None:
        if messages:
            print(f"Article {contentId} not found in the model.")
        return [], []

    # Get the vector for the article
    article_vector = X[article_index]

    # Use the pre-fitted model to find the nearest neighbors
    distances, indices = model.kneighbors(article_vector.reshape(1, -1), n_neighbors=k+1)
    distances = distances.flatten()
    indices = indices.flatten()

    # Exclude the first neighbor (which is the article itself)
    recommended_indices = indices[1:]
    recommended_distances = distances[1:]

    # Convert indices back to original content IDs
    recommended_contentIds = [item_inv_mapper[idx] for idx in recommended_indices]

    if messages:
        print("Recommended indices:", recommended_indices)
        print("Recommended content IDs:", recommended_contentIds)
        print("Distances:", recommended_distances)

    return recommended_contentIds, recommended_distances

In [19]:
# Number of recommendations per article
k = 5

# Create a dataframe whose index will be the article IDs (i.e. keys from item_mapper)
columns = ['title'] + [f"Recommendation {i+1}" for i in range(k)]
df_recommendations = pd.DataFrame(columns=columns, index=list(item_mapper.keys()))

# Make sure df_articles is indexed by 'contentId'
if df_movies_titles.index.name != 'show_id':
    df_movies_titles.set_index('show_id', inplace=True)

# Loop through each article in the model (each key in item_mapper)
for contentId in df_recommendations.index:
    # Get recommendations (list of recommended content IDs)
    rec_ids, rec_distances = recommend_articles(
        contentId, model, user_item_matrix, item_mapper, item_inv_mapper, k=k, messages=False
    )
    # Get the title of the article
    if contentId in df_movies_titles.index:
        df_recommendations.at[contentId, 'title'] = df_movies_titles.at[contentId, 'title']
    else:
        df_recommendations.at[contentId, 'title'] = 'Unknown Title'

    # Populate the recommendation columns with article titles
    for i, rec_id in enumerate(rec_ids):
        if rec_id in df_movies_titles.index:
            rec_title = df_movies_titles.at[rec_id, 'title']
        else:
            rec_title = 'Unknown Title'
        df_recommendations.at[contentId, f"Recommendation {i+1}"] = rec_title

# Display the recommendations dataframe
df_recommendations.head()

,title,Recommendation 1,Recommendation 2,Recommendation 3,Recommendation 4,Recommendation 5
1004,Zero,Atelier,Valentino,Who Killed Little Gregory?,Goodnight DJ 1,Zero
1005,Izzy's Koala World,Izzy's Koala World,Ninja Hattori,Game Over (Tamil Version),Legend Quest,Geronimo Stilton
1006,Keymon and Nani in Space Adventure,Animal Crackers,Super Monsters Furever Friends,Trolls Holiday Special,Latte and the Magic Waterstone,48 Christmas Wishes
1007,Motu Patlu Dino Invasion,Kahlil Gibran's The Prophet,Adore,Mr. Roosevelt,LEGO Marvel Super Heroes: Avengers Reassembled!,The Christmas Project
1018,Dhobi Ghat (Mumbai Diaries),Alejandro Sanz: What I Was Is What I Am,C Kkompany,Ghost in the Shell: SAC_2045,Ali & Ratu Ratu Queens,Ojukokoro: Greed


In [22]:
# Step 1: Helper function to get top N rated movies for a user
def get_top_rated_movies(user_df, user_id, top_n=3):
    user_ratings = user_df[user_df['user_id'] == user_id]
    return user_ratings.sort_values(by='rating', ascending=False).head(top_n)['show_id'].tolist()

# Step 2: Get recommendations for a given movie using the model
def get_similar_movies(movie_id, top_k=30):
    if movie_id not in item_mapper:
        return []
    movie_idx = item_mapper[movie_id]
    distances, indices = model.kneighbors(user_item_matrix[movie_idx], n_neighbors=top_k+1)
    similar_indices = indices.flatten()[1:]  # exclude the movie itself
    similar_movie_ids = [item_inv_mapper[i] for i in similar_indices]
    return similar_movie_ids

# Step 3: Create a fallback pool — most rated movies
top_global_movies = df_ratings['show_id'].value_counts().index.tolist()

# Step 4: Generate recommendations for all users
unique_users = df_ratings['user_id'].unique()
user_recommendations = {}

for user_id in unique_users:
    top_movies = get_top_rated_movies(df_ratings, user_id, top_n=3)
    recommended = []
    seen = set(df_ratings[df_ratings['user_id'] == user_id]['show_id'])

    for movie_id in top_movies:
        similar = get_similar_movies(movie_id, top_k=30)
        for rec_id in similar:
            if rec_id not in seen and rec_id not in recommended:
                recommended.append(rec_id)
            if len(recommended) >= 10:
                break
        if len(recommended) >= 10:
            break

    # Add fallback: most popular unseen movies
    for fallback in top_global_movies:
        if fallback not in seen and fallback not in recommended:
            recommended.append(fallback)
        if len(recommended) >= 10:
            break

    user_recommendations[user_id] = recommended[:10]


# Step 5: Convert to DataFrame
recommendation_df = pd.DataFrame.from_dict(user_recommendations, orient='index',
                                           columns=[f'rec_{i+1}' for i in range(10)])
recommendation_df.index.name = 'user_id'
recommendation_df.reset_index(inplace=True)

# Show result
recommendation_df.head()


,user_id,rec_1,rec_2,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10
0,1,6088,6112,61,6079,6101,6092,6199,6081,6116,6128
1,2,6092,61,6103,6077,6068,6054,6081,6079,6105,6110
2,3,6079,6081,6112,6111,6181,6103,6110,6088,61,6118
3,4,6081,6088,6118,61,6079,6110,6185,6112,6101,6138
4,5,5,7289,2255,473,2038,2175,1493,4858,5028,1433


First list. Gives 10 recommendations to user based on their top 3 rated movies

DEPLOYMENT

In [24]:
import sqlite3

# Step 1: Connect to (or create) a SQLite DB file
conn = sqlite3.connect('user_top10_recommendations.db')  # Creates a file in your current directory

# Step 2: Write the DataFrame to a table
recommendation_df.to_sql('user_recommendations', conn, if_exists='replace', index=False)

# Step 3: Close the connection
conn.close()

print("Export complete! SQLite DB saved as 'movie_recommendations.db'")


Export complete! SQLite DB saved as 'movie_recommendations.db'


Category Based recommendations!!

DATA PREPARATION

In [43]:
# reset dfs
df_movies_titles = pd.read_csv(r"C:\Users\camde\Downloads\movies_titles.csv")
df_user_ratings = pd.read_csv(r"C:\Users\camde\Downloads\movies_ratings.csv")

In [44]:
# re-run existing cleaning steps

df_movies_titles['show_id'] = df_movies_titles['show_id'].str.lstrip('s')
df_movies_titles.set_index('show_id', inplace=True)

df_user_ratings['show_id'] = df_user_ratings['show_id'].str.lstrip('s')
df_ratings = df_user_ratings.groupby(['user_id', 'show_id'], as_index=False)['rating'].max()


In [45]:
# re-build mappings and model
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

num_users = df_ratings['user_id'].nunique()
num_items = df_ratings['show_id'].nunique()

user_mapper = dict(zip(np.sort(df_ratings['user_id'].unique()), range(num_users)))
item_mapper = dict(zip(np.sort(df_ratings['show_id'].unique()), range(num_items)))
user_inv_mapper = {v: k for k, v in user_mapper.items()}
item_inv_mapper = {v: k for k, v in item_mapper.items()}

df_ratings['user_index'] = df_ratings['user_id'].map(user_mapper)
df_ratings['item_index'] = df_ratings['show_id'].map(item_mapper)

user_item_matrix = csr_matrix(
    (df_ratings['rating'], (df_ratings['item_index'], df_ratings['user_index'])),
    shape=(num_items, num_users)
)

model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
model.fit(user_item_matrix)


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [46]:
# Step 1: Identify genre columns (assuming they're 0/1 values)
genre_columns = [col for col in df_movies_titles.columns if df_movies_titles[col].dropna().isin([0,1]).all()]

# Step 2: Create a new 'listed_in' column by joining the genres where the value is 1
df_movies_titles['listed_in'] = df_movies_titles[genre_columns].apply(
    lambda row: ', '.join([genre for genre in genre_columns if row[genre] == 1]), axis=1
)

# Optional: Drop the original one-hot columns
df_movies_titles.drop(columns=genre_columns, inplace=True)

# Show result
df_movies_titles[['listed_in']].head()


,listed_in
show_id,
1,Documentaries
2,"Dramas, TV Dramas"
3,"Action, Adventure, TV Action"
4,"Docuseries, Reality TV"
5,"Comedies, TV Comedies"


MODELING AND EVALUATION

In [ ]:
# Step 1: Rebuild movie_to_genres mapping
df_movies_genres = df_movies_titles.reset_index()
df_movies_genres['genres'] = df_movies_genres['listed_in'].str.split(', ')
df_movies_genres = df_movies_genres.explode('genres')
movie_to_genres = df_movies_genres.groupby('show_id')['genres'].apply(set).to_dict()

# Step 2: Helper functions
def get_similar_movies(movie_id, top_k=30):
    if movie_id not in item_mapper:
        return []
    movie_idx = item_mapper[movie_id]
    distances, indices = model.kneighbors(user_item_matrix[movie_idx], n_neighbors=top_k+1)
    similar_indices = indices.flatten()[1:]
    similar_movie_ids = [item_inv_mapper[i] for i in similar_indices]
    return similar_movie_ids

def get_genre_filtered_recommendations(movie_id, user_seen, target_genre, top_k=30):
    recs = get_similar_movies(movie_id, top_k=top_k)
    genre_filtered = []
    for rec_id in recs:
        genres = movie_to_genres.get(rec_id, set())
        if target_genre in genres and rec_id not in user_seen and rec_id not in genre_filtered:
            genre_filtered.append(rec_id)
        if len(genre_filtered) >= 10:
            break
    return genre_filtered

def get_top_rated_movies(user_df, user_id, top_n=3):
    user_ratings = user_df[user_df['user_id'] == user_id]
    return user_ratings.sort_values(by='rating', ascending=False).head(top_n)['show_id'].tolist()


In [48]:
from collections import defaultdict

# Step 1: Get rating count per movie as a popularity proxy
movie_rating_counts = df_ratings['show_id'].value_counts()

# Step 2: Merge genre info and rating count
df_movie_genre_counts = df_movies_genres[['show_id', 'genres']].copy()
df_movie_genre_counts['rating_count'] = df_movie_genre_counts['show_id'].map(movie_rating_counts)

# Step 3: Build fallback dictionary for each genre
top_movies_by_genre = defaultdict(list)
for genre in df_movie_genre_counts['genres'].unique():
    top_in_genre = (
        df_movie_genre_counts[df_movie_genre_counts['genres'] == genre]
        .dropna(subset=['rating_count'])
        .sort_values(by='rating_count', ascending=False)
    )
    top_movies_by_genre[genre] = top_in_genre['show_id'].tolist()

# Step 4: Build recommendations with fallbacks for 5 test users
test_users = df_ratings['user_id'].unique()[:5]
all_genres = df_movies_genres['genres'].unique()
genre_recommendations_test = []

for user_id in test_users:
    user_seen = set(df_ratings[df_ratings['user_id'] == user_id]['show_id'])
    top_movies = get_top_rated_movies(df_ratings, user_id, top_n=3)

    for genre in all_genres:
        genre_recs = []

        for movie_id in top_movies:
            recs = get_genre_filtered_recommendations(movie_id, user_seen, genre, top_k=50)
            genre_recs.extend(recs)
            if len(genre_recs) >= 10:
                break

        # De-duplicate and fill with fallback popular movies from same genre
        genre_recs = list(dict.fromkeys(genre_recs))
        for fallback_id in top_movies_by_genre[genre]:
            if fallback_id not in genre_recs and fallback_id not in user_seen:
                genre_recs.append(fallback_id)
            if len(genre_recs) >= 10:
                break

        genre_recs = genre_recs[:10]
        genre_recommendations_test.append([user_id, genre] + genre_recs)

# Step 5: Convert to DataFrame
genre_rec_columns = ['user_id', 'genre'] + [f'rec_{i+1}' for i in range(10)]
genre_rec_df_test = pd.DataFrame(genre_recommendations_test, columns=genre_rec_columns)

# Preview
genre_rec_df_test.head()


,user_id,genre,rec_1,rec_2,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10
0,1,Documentaries,6293,6150,6105,6111,6128,8276,7982,2867,263,7131
1,1,Dramas,6068,6185,6251,6278,6290,6284,6281,6158,6092,6112
2,1,TV Dramas,6068,6251,6138,6157,5745,2179,3282,6508,1515,3852
3,1,Action,6247,6245,6088,61,6103,619,62,7864,8057,540
4,1,Adventure,6247,6245,6088,61,6103,619,62,7864,8057,540


table where we can get a user's id and top 10 recommendations for each genre according to the top 3 movie's they've rated. We ran into the issue of not having enough user ratings for each genre so in the case there wasn't enough, the recommender is built to instead suggest the top rated shows in that genre

DEPLOYMENT

In [42]:
# export to sqlite

import sqlite3

# Step 1: Connect to (or create) a SQLite DB file
conn = sqlite3.connect('user_genre_recommendations.db')  # File will be created in your current working directory

# Step 2: Export the DataFrame to a table
genre_rec_df_test.to_sql('user_genre_recommendations', conn, if_exists='replace', index=False)

# Step 3: Close the connection
conn.close()

print("✅ Exported to 'genre_recommendations.db' successfully!")


✅ Exported to 'genre_recommendations.db' successfully!
